In [1]:
# Importing the necessary packages
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS, Chroma
from langchain.chains import RetrievalQA
import textwrap

In [2]:
# Load multiple PDFs into a list of documents
def load_pdfs(file_paths):
    all_docs = []
    for file_path in file_paths:
        loader = PyMuPDFLoader(file_path=file_path)
        docs = loader.load()
        all_docs.extend(docs)  # Add documents from each PDF to the combined list
    return all_docs

In [3]:
# Responsible for splitting the documents into several chunks
def split_docs(documents, chunk_size=1000, chunk_overlap=20):

    # Initializing the RecursiveCharacterTextSplitter with
    # chunk_size and chunk_overlap
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )

    # Splitting the documents into chunks
    chunks = text_splitter.split_documents(documents=documents)

    # returning the document chunks
    return chunks

In [4]:
# function for loading the embedding model
def load_embedding_model(model_path, normalize_embedding=True):
    return HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs={'device':'cpu'}, # here we will run the model with CPU only
        encode_kwargs = {
            'normalize_embeddings': normalize_embedding # keep True to compute cosine similarity
        }
    )

In [5]:
# Function for creating embeddings using FAISS
def create_embeddings_faiss(chunks, embedding_model, storing_path="vectorstore"):
    # Creating the embeddings using FAISS
    vectorstore = FAISS.from_documents(chunks, embedding_model, storing_path)

    # returning the vectorstore
    return vectorstore

In [6]:
def create_embeddings_chroma(chunks, embedding_model, storing_path="chroma_store"):
    # Create the Chroma vectorstore
    vectorstore = Chroma.from_documents(
        chunks,
        embedding_model
    )
    
    # Persist the Chroma vectorstore to disk
    vectorstore.persist(storing_path)

    return vectorstore


In [7]:
# Creating the chain for Question Answering
def load_qa_chain(retriever, llm, prompt):
    return RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever, # here we are using the vectorstore as a retriever
        chain_type="stuff",
        return_source_documents=True, # including source documents in output
        chain_type_kwargs={'prompt': prompt} # customizing the prompt
    )

In [8]:
# Prettifying the response
def get_response(query, chain):
    # Getting response from chain
    response = chain.invoke({'query': query})
    
    # Wrapping the text for better output in Jupyter Notebook
    wrapped_text = textwrap.fill(response['result'], width=100)
    print(response['result'])

## Implementation

In [9]:
from langchain_ollama.llms import OllamaLLM
from langchain import PromptTemplate

In [10]:
# llm = OllamaLLM(model="orca-mini", temperature=0)

In [11]:
llm = OllamaLLM(model="llama3.2", temperature=0)

In [12]:
# llm = OllamaLLM(model="mistral", temperature=0)

In [13]:
# embed = load_embedding_model(model_path="all-MiniLM-L6-v2")
embed = load_embedding_model(model_path="sentence-transformers/all-mpnet-base-v2")
# embed = load_embedding_model(model_path="sentence-transformers/multi-qa-mpnet-base-dot-v1")

In [14]:
file_paths = [
    "./docs/rodtep_guidelines-output.pdf",
    "./docs/Drawback-Rates.pdf",
    "./docs/furniture_tv.pdf",
]
docs = load_pdfs(file_paths=file_paths)

In [15]:
documents = split_docs(documents=docs)

In [16]:
# creating vectorstore
# vectorstore = create_embeddings_faiss(documents, embed)
vectorstore = create_embeddings_chroma(documents, embed)

# converting vectorstore to a retriever
retriever = vectorstore.as_retriever()

/tmp/ipykernel_1058442/1798578560.py:9: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist(storing_path)


TypeError: Chroma.persist() takes 1 positional argument but 2 were given

In [ ]:
# Creating the prompt from the template which we created before
prompt = PromptTemplate.from_template("""
### System:
You are an respectful and honest assistant. You have to answer the user's \
questions using only the context provided to you. If you don't know the answer, \
please think rationally and answer from your own knowledge base

### Context:
{context}

### User:
{question}

### Response:
""")

# Creating the chain
chain = load_qa_chain(retriever, llm, prompt)

In [ ]:
get_response("Explain the Kaplan–Meier Survival Curve", chain)

In [ ]:
get_response("Explain the RDOTEP Guidelines", chain)

In [ ]:
get_response("What are some of the things I have to keep in mind while exporting electronics", chain)

In [ ]:
get_response("What if i want to export a apple iphone mobile to USA", chain)

In [ ]:
get_response("What if i want to export furniture", chain)

In [ ]:
get_response("Give me a checklist if i want to export furniture", chain)

In [ ]:
get_response("What are the rates?", chain)